# Script to create seed data

This script will create a seed data of users, products, and orders and save them as CSV files
Why CSV files? As the focus is analytical functions on Python, creating a queryable DB isn't really worth it. Plus, I'm lazy.

In [1]:
import pandas as pd # Will be converting the data into dataframes and exporting to csv form
from random import random, randrange # To randomly generate different prices, and IDs

# The filenames to be used
USERS = 'users.csv'
PRODUCTS = 'products.csv'
ORDERS = 'orders.csv'

In [2]:
# A function to generate a randome date between 2018-01-01 and 2019-12-29 and time between 09:00:00 and 22:59:59
def generate_random_time(year=None, hour=None, month=None):
    d = "{:02d}".format(randrange(1, 30))
    if not month:
        m = "{:02d}".format(randrange(1, 13))
    else:
        m = "{:02d}".format(int(month))
    if not year:
        y = str(randrange(2018, 2020))
    else:
        y = year
    if not hour:
        H = "{:02d}".format(randrange(0, 24))
    else:
        H = "{:02d}".format(int(hour))
    M = "{:02d}".format(randrange(0, 60))
    S = "{:02d}".format(randrange(0, 60))
    return '-'.join([y, m, d])+' '+':'.join([H, M, S]) # Returns in the format "yyyy-mm-dd hh:mm:ss"

In [3]:
# How many different users and cities
user_count = 1000
city_count = 25
users = {
    'uid': [],
    'uname': [],
    'city': []
}

In [4]:
# How many different products and categories
product_count = 15
category_count = 4
products = {
    'pid': [],
    'pname': [],
    'category': [],
    'price': []
}

In [5]:
# Min & Max orders per person
orders = {
    'oid': [],
    'pid': [],
    'qty': [],
    'uid': [],
    'date': [],
    'total': [],
}

In [6]:
# Generate users
for i in range(1, user_count+1):
    users['uid'].append(i)
    users['uname'].append('User '+str(i))
    users['city'].append('City '+str(randrange(1, city_count+1)))
users = pd.DataFrame(users)

In [7]:
# Generate products
for i in range(1, product_count+1):
    products['pid'].append(i)
    products['pname'].append('Product '+str(i))
    products['category'].append('Category '+str(randrange(1, category_count+1)))
    products['price'].append(round(random()*100,2))
products = pd.DataFrame(products)

In [8]:
# Generate orders
def create_orders(oc, min_o, max_o, users, products, orders, year=None, month=None, hour=None):
    for i, row in users.iterrows():
        uid = row['uid']
        no_of_orders = randrange(min_o, max_o+1)
        for j in range(no_of_orders):
            p = randrange(products['pid'].min(), products['pid'].max())
            price = float(products[products.pid == p]['price'])
            qty = randrange(1,5)
            total = price*qty
            orders['oid'].append(oc)
            oc+=1
            orders['uid'].append(uid)
            orders['pid'].append(p)
            orders['qty'].append(qty)
            orders['date'].append(generate_random_time(year=year, hour=hour, month=month))
            orders['total'].append(total)
    return orders, oc

Concept for generating orders:

While iterating through each user we've created, we are randomly total number of orders for that user between 20 and 40, and then we create that many orders with a random product from `products` with a random quantity between 1 and 4

To add a bit of realism to the data, we'll be 


In [9]:
# Generate orders
order_counter = 0
for mm in range(1, 13):
    orders, order_counter = create_orders(order_counter, 20+2*mm, 25+2*mm, users, products, orders, month=mm)
    
orders, order_counter = create_orders(order_counter, int(5), 
                                      int(15), users, products, orders, year='2019')
for hh in range(10, 18):
    orders, order_counter = create_orders(order_counter, int(25-2*abs(hh-14)), 
                                          int(30-2*abs(hh-14)), users, products, orders, hour=str(hh))


In [10]:
orders = pd.DataFrame(orders)

In [11]:
# Write them out to a file
users.to_csv(USERS, header=True)
products.to_csv(PRODUCTS, header=True)
orders.to_csv(ORDERS, header=True)

### Testing

In [12]:
orders.merge(products, left_on='pid', right_on='pid', how='left').merge(users, left_on='uid', right_on='uid', how='left')

,oid,pid,qty,uid,date,total,pname,category,price,uname,city
0,0,13,2,1,2019-01-26 00:32:50,159.50,Product 13,Category 1,79.75,User 1,City 21
1,1,1,3,1,2019-01-25 03:45:10,158.85,Product 1,Category 3,52.95,User 1,City 21
2,2,5,4,1,2018-01-11 00:03:37,16.96,Product 5,Category 4,4.24,User 1,City 21
3,3,1,3,1,2019-01-15 15:11:29,158.85,Product 1,Category 3,52.95,User 1,City 21
4,4,9,1,1,2019-01-24 18:26:10,31.29,Product 9,Category 1,31.29,User 1,City 21
...,...,...,...,...,...,...,...,...,...,...,...
624146,624146,8,4,1000,2018-03-24 17:43:30,230.56,Product 8,Category 2,57.64,User 1000,City 10
624147,624147,4,3,1000,2018-07-10 17:39:19,29.76,Product 4,Category 1,9.92,User 1000,City 10
624148,624148,5,2,1000,2019-07-01 17:04:48,8.48,Product 5,Category 4,4.24,User 1000,City 10
624149,624149,14,3,1000,2018-04-23 17:02:55,125.28,Product 14,Category 4,41.76,User 1000,City 10


In [13]:
len(pd.DataFrame(orders))

624151